## Playground for Poincare sphere interactive plots
#### For the project "Representing Mueller Matrices as Geometric Transformations on the Poincare Sphere"
Kate Salesin, 12/4/20

In [14]:
import math
import numpy as np
import numpy.linalg as la
import plotly.offline as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, interact

from mueller import *

In [15]:
def eval_stokes(M, s1, s2, s3):
    n = s1.shape[0]
    s1_ = np.zeros(n)
    s2_ = np.zeros(n)
    s3_ = np.zeros(n)
    
    for i in range(n):
        S = np.array([[1.],[s1[i]],[s2[i]],[s3[i]]])
        S_out = M @ S
#         S_out /= la.norm(S_out)

        s1_[i] = S_out[1]
        s2_[i] = S_out[2]
        s3_[i] = S_out[3]
        
    return (s1_, s2_, s3_)

In [16]:
def poincare_mesh(mesh = True, boundary = True):
    # Sphere grid
    n = 20
    U, V = np.meshgrid(np.linspace(0, math.pi, n), np.linspace(0, 2 * math.pi, n))

    if boundary:
        X = (np.cos(U) * np.sin(V)).flatten()
        Y = (np.sin(U) * np.sin(V)).flatten()
        Z = (np.cos(V)).flatten()
        sz = Z.shape[0]
    
        sph = go.Scatter3d(
            x = X, y = Y, z = Z,
            mode = 'markers',
            marker = dict(
                size = 3,
                color = Z,
                colorscale = 'Viridis',
                opacity = 0.5
            )
        )

        viridis = np.array([[0.267004, 0.004874, 0.329415, 1.      ],
                           [0.282623, 0.140926, 0.457517, 1.      ],
                           [0.253935, 0.265254, 0.529983, 1.      ],
                           [0.206756, 0.371758, 0.553117, 1.      ],
                           [0.163625, 0.471133, 0.558148, 1.      ],
                           [0.127568, 0.566949, 0.550556, 1.      ],
                           [0.134692, 0.658636, 0.517649, 1.      ],
                           [0.266941, 0.748751, 0.440573, 1.      ],
                           [0.477504, 0.821444, 0.318195, 1.      ],
                           [0.741388, 0.873449, 0.149561, 1.      ],
                           [0.993248, 0.906157, 0.143936, 1.      ]])

        vertexcolors = []

        for i, z in enumerate(Z):
            base = (z + 1) * 5
            idx = min(int(base), 9)
            t = base - idx

            c0 = viridis[idx]
            c1 = viridis[idx + 1]

            col = (1-t) * c0 + t * c1
            col *= 255

            col_str = 'rgba(' + str(int(col[0])) + ',' + str(int(col[1])) + ',' + str(int(col[2])) + ',1)'
            vertexcolors.append(col_str)

        if mesh:
            out = go.Mesh3d(
                x = [], y = [], z = [],
                alphahull = 0,
                vertexcolor = vertexcolors,
                opacity = 0.4,
                flatshading = True
            )
        else:
            out = go.Scatter3d(
                x = [], y = [], z = [],
                mode = 'lines',
                line = dict(
                    width = 5,
                    color = 'rgba(255,255,0,0.8)'
                )
            )

    else:
        X = 0.5 * (np.cos(U) * np.sin(V)).flatten()
        Y = 0.5 * (np.sin(U) * np.sin(V)).flatten()
        Z = 0.5 * (np.cos(V)).flatten()
        sz = Z.shape[0]

        sph = go.Scatter3d(
            x = X, y = Y, z = Z,
            mode = 'markers',
            marker = dict(
                size = 3,
                color = Z,
                colorscale = 'Plotly3',
                opacity = 0.5
            )
        )

        plasma = np.array([[0.        , 1.        , 1.        , 1.        ],
                           [0.09803922, 0.90196078, 1.        , 1.        ],
                           [0.2       , 0.8       , 1.        , 1.        ],
                           [0.29803922, 0.70196078, 1.        , 1.        ],
                           [0.4       , 0.6       , 1.        , 1.        ],
                           [0.50196078, 0.49803922, 1.        , 1.        ],
                           [0.6       , 0.4       , 1.        , 1.        ],
                           [0.70196078, 0.29803922, 1.        , 1.        ],
                           [0.8       , 0.2       , 1.        , 1.        ],
                           [0.90196078, 0.09803922, 1.        , 1.        ],
                           [1.        , 0.        , 1.        , 1.        ]])

        vertexcolors = []

        for i, z in enumerate(Z):
            base = (z + 1) * 5
            idx = min(int(base), 9)
            t = base - idx

            c0 = plasma[idx]
            c1 = plasma[idx + 1]

            col = (1-t) * c0 + t * c1
            col *= 255

            col_str = 'rgba(' + str(int(col[0])) + ',' + str(int(col[1])) + ',' + str(int(col[2])) + ',1)'
            vertexcolors.append(col_str)

        if mesh:
            out = go.Mesh3d(
                x = [], y = [], z = [],
                alphahull = 0,
                vertexcolor = vertexcolors,
                opacity = 0.4,
                flatshading = True
            )
        else:
            out = go.Scatter3d(
                x = [], y = [], z = [],
                mode = 'lines',
                line = dict(
                    width = 5,
                    color = 'rgba(255,255,0,0.8)'
                )
            )

    s1_ref = go.Scatter3d(
        x = [0, 1], y = [0, 0], z = [0, 0],
        mode = 'lines',
        line = dict(
            width = 5,
            color = 'rgba(255,0,0,0.5)'
        )
    )

    s2_ref = go.Scatter3d(
        x = [0, 0], y = [0, 1], z = [0, 0],
        mode = 'lines',
        line = dict(
            width = 5,
            color = 'rgba(0,255,0,0.5)'
        )
    )

    s3_ref = go.Scatter3d(
        x = [0, 0], y = [0, 0], z = [0, 1],
        mode = 'lines',
        line = dict(
            width = 5,
            color = 'rgba(0,0,255,0.5)'
        )
    )

    # Additional output variable
    aov = go.Scatter3d(
        x = [], y = [], z = [],
        mode = 'lines',
        line = dict(
            width = 5,
            color = 'rgba(255,0,255,0.8)'
        )
    )

    scene_axis_layout = dict(
        color='white',
        backgroundcolor='rgba(0,0,0,0)',
        gridcolor='rgba(0,0,0,0.1)'
    )

    layout = go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=0, r=0, b=0, t=0),
        scene_xaxis = scene_axis_layout,
        scene_yaxis = scene_axis_layout,
        scene_zaxis = scene_axis_layout,
        scene_xaxis_title = 's1',
        scene_yaxis_title = 's2',
        scene_zaxis_title = 's3',
        showlegend=False
    )

    fig = go.FigureWidget(data=[sph, s1_ref, s2_ref, s3_ref, aov, out], layout=layout)

    return (fig, X, Y, Z)

In [25]:
fig, X, Y, Z = poincare_mesh(mesh=True, boundary=True)

@interact
def update(scale = widgets.FloatSlider(min = 0., max = 1., step = 0.01, value = 1., 
                                   layout = widgets.Layout(width = '75%'))):
    Me = np.eye(4) * scale
    print(Me)
    
    X_, Y_, Z_ = eval_stokes(Me, X, Y, Z)
    
    with fig.batch_update():
        fig.data[-1].x = X_
        fig.data[-1].y = Y_
        fig.data[-1].z = Z_
        
fig

interactive(children=(FloatSlider(value=1.0, description='scale', layout=Layout(width='75%'), max=1.0, step=0.…

FigureWidget({
    'data': [{'marker': {'color': array([1., 1., 1., ..., 1., 1., 1.]),
                       …

### Guide to interactive figure:
* Dots = input Stokes vectors
* Mesh = output Stokes vectors
* Red axis = $s_1$ axis (H/V linear polarization)
* Green axis = $s_2$ axis (45/135 linear polarization)
* Blue axis = $s_3$ axis (R/L circular polarization)

Above plot: Mueller matrix producing output vectors

In [18]:
fig, X, Y, Z = poincare_mesh(mesh=False, boundary=True)

# fig.write_html("plots/fig1.html", include_plotlyjs='directory')

@interact
def update(angle = widgets.IntSlider(min = 0, max = 180, step = 1, value = 0, 
                                   layout = widgets.Layout(width = '75%'))):
    
    Mt = polarizer_mueller(np.deg2rad(angle))
    print(Mt)
    
    X_, Y_, Z_ = eval_stokes(Mt, X, Y, Z)
    
    with fig.batch_update():
        fig.data[-1].x = X_
        fig.data[-1].y = Y_
        fig.data[-1].z = Z_
        
fig

interactive(children=(IntSlider(value=0, description='angle', layout=Layout(width='75%'), max=180), Output()),…

FigureWidget({
    'data': [{'marker': {'color': array([1., 1., 1., ..., 1., 1., 1.]),
                       …

#### Fig. 1. A linear polarizer maps any input $\vec{s}$ (outer sphere markers) to some point on the diattenuation vector $\hat{D}$. The output values are all on the yellow line, which coincides with $\hat{D}$.

In [19]:
fig, X, Y, Z = poincare_mesh(mesh=True, boundary=True)

@interact
def update(D = widgets.FloatSlider(min = 0.01, max = 1., step = 0.005, value = 0.5, 
                                   layout = widgets.Layout(width = '75%')),
           elevation = widgets.IntSlider(min = 0, max = 90, step = 1, value = 0, 
                                   layout = widgets.Layout(width = '75%')),
           azimuth = widgets.IntSlider(min = 0, max = 360, step = 5, value = 60, 
                                   layout = widgets.Layout(width = '75%'))):
    theta = math.pi / 2 - np.deg2rad(elevation)
    phi = np.deg2rad(azimuth)
    
    x_ = D * math.sin(theta) * math.cos(phi)
    y_ = D * math.sin(theta) * math.sin(phi)
    z_ = D * math.cos(theta)
    D_ = 1 - 0.5 * math.sqrt(D)
    
    Md = diattenuator_mueller(D_, *(D_ * np.array([x_, y_, z_])))
    print(Md)
    
    X_, Y_, Z_ = eval_stokes(Md, X, Y, Z)
    
    with fig.batch_update():
        fig.data[-1].x = X_
        fig.data[-1].y = Y_
        fig.data[-1].z = Z_
        
        fig.data[-2].x = [0, x_]
        fig.data[-2].y = [0, y_]
        fig.data[-2].z = [0, z_]
        
fig

interactive(children=(FloatSlider(value=0.5, description='D', layout=Layout(width='75%'), max=1.0, min=0.01, s…

FigureWidget({
    'data': [{'marker': {'color': array([1., 1., 1., ..., 1., 1., 1.]),
                       …

#### Figure 2. For the general case $0 < D < 1$, a diattenuator maps input $\vec{s}$ somewhere on the plane spanned by $\vec{s}$ and $\hat{D}$.

In [20]:
fig, X, Y, Z = poincare_mesh(mesh=True, boundary=True)

@interact
def update(phase_shift = widgets.IntSlider(min = 0, max = 180, step = 1, value = 0, 
                                     layout = widgets.Layout(width = '75%')),
           theta = widgets.IntSlider(min = 0, max = 180, step = 1, value = 0, 
                                     layout = widgets.Layout(width = '75%'),
                                     description = 'rotation angle of waveplate')):
    
    Mr = waveplate_mueller(np.deg2rad(phase_shift), np.deg2rad(theta))
    print(Mr)
    
    X_, Y_, Z_ = eval_stokes(Mr, X, Y, Z)
    
    with fig.batch_update():
        fig.data[-1].x = X_
        fig.data[-1].y = Y_
        fig.data[-1].z = Z_
        
fig

interactive(children=(IntSlider(value=0, description='phase_shift', layout=Layout(width='75%'), max=180), IntS…

FigureWidget({
    'data': [{'marker': {'color': array([1., 1., 1., ..., 1., 1., 1.]),
                       …

#### Figure 3. A waveplate is a simple axis-angle rotation. Notice when the phase shift = $90^{\circ}$ (a quarter-wave plate), $45^{\circ}$ linear polarization becomes right circular polarization, as we would expect.

In [21]:
fig, X, Y, Z = poincare_mesh(mesh=True, boundary=True)

@interact
def update(alpha = widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = 1, 
                                       layout = widgets.Layout(width = '75%')),
           beta = widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = 1, 
                                       layout = widgets.Layout(width = '75%')),
           gamma = widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = 1, 
                                       layout = widgets.Layout(width = '75%'))):
    
    Mt = depolarizer_mueller(alpha, beta, gamma)
    print(Mt)
    
    X_, Y_, Z_ = eval_stokes(Mt, X, Y, Z)
    
    with fig.batch_update():
        fig.data[-1].x = X_
        fig.data[-1].y = Y_
        fig.data[-1].z = Z_
        
fig

interactive(children=(FloatSlider(value=1.0, description='alpha', layout=Layout(width='75%'), max=1.0, min=0.0…

FigureWidget({
    'data': [{'marker': {'color': array([1., 1., 1., ..., 1., 1., 1.]),
                       …

#### Figure 4. A depolarizer acts as non-uniform scale along three orthogonal axes.

In [22]:
fig, X, Y, Z = poincare_mesh(mesh=True, boundary=True)

@interact
def update(theta = widgets.FloatSlider(min = 0, max = 90, step = 1, value = 0, 
                                       layout = widgets.Layout(width = '75%')),
           eta = widgets.FloatSlider(min = 1, max = 3, step = 0.01, value = 1.33, 
                                       layout = widgets.Layout(width = '75%'))):
    
    Mf = specular_transmission_mueller(math.cos(np.deg2rad(theta)), eta)
    print(Mf)
    
    X_, Y_, Z_ = eval_stokes(Mf, X, Y, Z)
    
    with fig.batch_update():
        fig.data[-1].x = X_
        fig.data[-1].y = Y_
        fig.data[-1].z = Z_
        
fig

interactive(children=(FloatSlider(value=0.0, description='theta', layout=Layout(width='75%'), max=90.0, step=1…

FigureWidget({
    'data': [{'marker': {'color': array([1., 1., 1., ..., 1., 1., 1.]),
                       …

#### Figure 5. Specular transmission decreases as the angle of incidence increases.

In [23]:
fig, X, Y, Z = poincare_mesh(mesh=True, boundary=True)

@interact
def update(theta = widgets.FloatSlider(min = 0, max = 90, step = 1, value = 0, 
                                       layout = widgets.Layout(width = '75%')),
           eta = widgets.FloatSlider(min = 1, max = 3, step = 0.01, value = 1.33, 
                                       layout = widgets.Layout(width = '75%'))):
    
    Mf = specular_reflection_mueller(math.cos(np.deg2rad(theta)), eta)
    print(Mf)
    
    X_, Y_, Z_ = eval_stokes(Mf, X, Y, Z)
    
    with fig.batch_update():
        fig.data[-1].x = X_
        fig.data[-1].y = Y_
        fig.data[-1].z = Z_
        
fig

interactive(children=(FloatSlider(value=0.0, description='theta', layout=Layout(width='75%'), max=90.0, step=1…

FigureWidget({
    'data': [{'marker': {'color': array([1., 1., 1., ..., 1., 1., 1.]),
                       …

#### Figure 6. Specular reflection increases as the angle of incidence increases and creates an increasing amount of linear polarization toward Brewster's angle (around $\theta = 53^{\circ}$ with the default $\eta$).